In [ ]:
pip install textblob


In [ ]:
import requests
import csv
from datetime import datetime
from textblob import TextBlob

# NewsAPI key
api_key = 'c7ed55456f4d44828f356288cffb68e3'

# endpoint and params
url = 'https://newsapi.org/v2/everything'

query = 'avian influenza'
page_size = 10  # max articles per request
total_pages = 10  # max pages to fetch

def sentiment_description(score):
    if score > 0.2:
        return "Positive"
    elif score < -0.2:
        return "Negative"
    else:
        return "Neutral"

# open csv file for writing
with open('news_articles.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # write the header row
    writer.writerow(['Post_id', 'Source', 'Title', 'Author', 'Timestamp', 'Text', 'Post_URL', 'Title_Sentiment_Score', 'Title_Sentiment', 'Text_Sentiment_Score', 'Text_Sentiment'])

    for page in range(1, total_pages + 1):
        params = {
            'apiKey': api_key,
            'q': query,
            'pageSize': page_size,
            'page': page
        }

        # make the request
        response = requests.get(url, params=params)

        # check for successful response
        if response.status_code == 200:
            data = response.json()
            articles = data['articles']

            # write article data rows
            for i, article in enumerate(articles, start=(page - 1) * page_size + 1):
                title = article['title']
                text = article.get('content', '')

                # perform sentiment analysis
                title_sentiment_score = TextBlob(title).sentiment.polarity
                text_sentiment_score = TextBlob(text).sentiment.polarity if text else 0

                title_sentiment = sentiment_description(title_sentiment_score)
                text_sentiment = sentiment_description(text_sentiment_score)

                writer.writerow([
                    f'news_{i}',  # post id (generic unique identifier)
                    article['source']['name'],  # source
                    title,  # title
                    article['author'] if article['author'] else 'Unknown',  # author
                    datetime.strptime(article['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%m/%d/%Y %I:%M:%S %p %Z"),  # timestamp
                    text,  # text
                    article['url'],  # post url
                    title_sentiment_score,  # title sentiment score
                    title_sentiment,  # title sentiment
                    text_sentiment_score,  # text sentiment score
                    text_sentiment  # text sentiment
                ])
        else:
            print(f"Error: {response.status_code}")
            break  # stop if there's an error

print('CSV file with sentiment analysis and descriptions has been created successfully.')


CSV file with sentiment analysis and descriptions has been created successfully.
